In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

from sklearn import metrics
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import datetime
import time
from datetime import timedelta

import os
import pickle
import requests
import json
import flask

%matplotlib inline

In [2]:
df = pd.read_csv(r"C:\Users\rg654th\Downloads\Flight Delay\Clean Data\Clean Historical Data.csv", parse_dates=[0], infer_datetime_format=True)

In [3]:
# New Column and Format
df['Scheduled DateTime'] = pd.to_datetime(df['Scheduled DateTime'], infer_datetime_format=True)
df['Scheduled Time'] = df['Scheduled DateTime'].dt.strftime('%H:00')
df['Delay? (20 min)'] = np.where(df['Minute Delay'] >= 20, 'Yes', 'No')
df.head(1)

,Terminal,Destination,Airline,Flight No.,Scheduled DateTime,Month,Weekday,Minute Delay,Delay Type,Scheduled Date,Scheduled Time,Real Departure Time,Delay? (20 min)
0,T2,Glasgow,Aer Lingus,EI3228,2019-05-31 18:30:00,May,Friday,14,Short,31-05-2019,18:00,18:44,No


In [4]:
traindf = df.drop(['Flight No.', 'Delay Type', 'Minute Delay', 'Month', 'Scheduled Date','Scheduled DateTime', 'Real Departure Time'], axis=1)
traindf = traindf.dropna()
traindf['Origin'] = "Dublin"
traindf.head(3)

,Terminal,Destination,Airline,Weekday,Scheduled Time,Delay? (20 min),Origin
0,T2,Glasgow,Aer Lingus,Friday,18:00,No,Dublin
1,T2,Alicante,Aer Lingus,Friday,18:00,Yes,Dublin
2,T1,Vienna,Laudamotion,Friday,18:00,No,Dublin


In [5]:
# getting airline, weeekday and destination list

list_airlines = traindf.Airline.unique()
list_airlines.sort()
list_destination = traindf.Destination.unique()
list_destination.sort()
# weekdays = Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday

In [6]:
newtraindf = pd.get_dummies(traindf)
newtraindf.head(1)

,Terminal_T1,Terminal_T2,Destination_Aberdeen,Destination_Abu Dhabi,Destination_Agadir,Destination_Alicante,Destination_Almeria,Destination_Amsterdam,Destination_Athens,Destination_Atlanta,...,Scheduled Time_17:00,Scheduled Time_18:00,Scheduled Time_19:00,Scheduled Time_20:00,Scheduled Time_21:00,Scheduled Time_22:00,Scheduled Time_23:00,Delay? (20 min)_No,Delay? (20 min)_Yes,Origin_Dublin
0,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,1


In [7]:
# Split Data
x_data = newtraindf.drop(['Delay? (20 min)_No', 'Delay? (20 min)_Yes'],axis=1)
y_data = newtraindf['Delay? (20 min)_Yes']
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data,test_size=0.26)

In [8]:
# Ensemble 
from sklearn.ensemble import AdaBoostClassifier

C:\Users\rg654th\AppData\Local\Continuum\anaconda3\envs\tfdeeplearning\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [9]:
adamodel = AdaBoostClassifier()
adamodel.fit(x_train, y_train)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None)

In [10]:
y_pred = adamodel.predict(x_test)

score2 = metrics.accuracy_score(y_test, y_pred)
score3 = metrics.precision_score(y_test, y_pred)
score4 = metrics.recall_score(y_test, y_pred)

print("Precision", "", score3)
print("Accuracy", "", score2)
print("Recall", "", score4)

Precision  0.6212121212121212
Accuracy  0.689928628072958
Recall  0.28018223234624146


In [11]:
with open('model.pkl', 'wb') as fid:
    pickle.dump(adamodel, fid,2) 

In [12]:
# get dictionary for reference later in form
index_dict = dict(zip(x_data.columns,range(x_data.shape[1])))

In [13]:
with open('x_data', 'wb') as fid:
    pickle.dump(index_dict, fid,2)  